### Selecting an Ideal site for opening a new rock-climbing gym in Denver, CO.

Introduction / Business problem

The summertime weather in CO is wonderful.  Sunny warm days and crisp cool nights.  The mountains and foothills all over the Front Range have their rock faces covered with climbers eagerly searching for the next foothold.  But the winters can be long and cold.  A solution to the  bitter winds and frost preventing an intrepid climber from perching on their favorite peak is to put the rock ledge inside a big warm building.  Indeed, several indoor rock-climbing gyms can currently be found scattered along the Front Range.  But none of them are your rock-climbing gym.  We want to show you exactly where your dream gym should be opened.  And what neighborhood site will provide the highest return on your investment.  Nothing is a sure thing, and all ventures carry risk.  We are living in a time when these risks can be calculated and mitigated by powerful technologies and machine learning algorithms.  We are going to provide you with an array of possible locations that have the maximum potential of financial success and longevity.


Data

In order to facilitate this process, we will be using FourSquare’s vast collection of data.  The actions and comments of thousands of users are collected and stored for access by our algorithms.  Countless actual reactions to myriad venues, events, and experiences can provide a backdrop of information for us to base our calculations on.  Every currently operating venue in the city will be provided for an exact count of gyms, health clubs, yoga studios, and especially other climbing gyms in the vicinities.  The likes and preferences of the clients that patronize these venues are also collected and will allow us to provide actionable insights into not only the locations, but also the clientele in those locations and their personal preferences.  This information will allow you to open your climbing gym in a location that suites your style, and also allow you to operate in a way that will cater to your local clients.  

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup 
import requests  
import seaborn as sns

In [2]:
#url =  'statistical_neighborhoods'
df = pd.read_csv('statistical_neighborhoods.csv')

df.head()

,Borough,Neighborhood,Latitude,Longitude
0,River North,Auraria,39.7506,-105.0075
1,Capitol Hill,Cherry Creek,39.7195,-104.9481
2,Capitol Hill,Congress Park,39.7329,-104.9494
3,Lakeside,West Highland,39.7647,-105.0407
4,Lakeside,Sloan Lake,39.7486,-105.0473


In [3]:
df[["Latitude", "Longitude"]] = df[["Latitude", "Longitude"]].astype("float")

In [4]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,River North,Auraria,39.7506,-105.0075
1,Capitol Hill,Cherry Creek,39.7195,-104.9481
2,Capitol Hill,Congress Park,39.7329,-104.9494
3,Lakeside,West Highland,39.7647,-105.0407
4,Lakeside,Sloan Lake,39.7486,-105.0473


In [5]:
address = 'Denver'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Denver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Denver are 39.7392364, -104.9848623.


In [6]:
# create map using latitude and longitude values
map_denver = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_denver)  
    
#map_denver

In [7]:
CLIENT_ID = 'LHZARKWYYEMMNULRDWIRLIVOF5RICOU5HZSXOCOLCLTWA3X3' # your Foursquare ID
CLIENT_SECRET = 'QGAASYMBSDJFOK5VX4IM4C1M22A43FJYSRUVQC1VU2234BLI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LHZARKWYYEMMNULRDWIRLIVOF5RICOU5HZSXOCOLCLTWA3X3
CLIENT_SECRET:QGAASYMBSDJFOK5VX4IM4C1M22A43FJYSRUVQC1VU2234BLI


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
# type your answer here

denver_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Auraria
Cherry Creek
Congress Park
West Highland
Sloan Lake
Berkeley
Chaffee Park
Highland
Wellshire
Montclair
South Park Hill
University Park
Hampden
Indian Creek
University Hills
Harvey Park
Sun Valley
Valverde
Villa Park
Five Points
Bear Valley
Virginia Village
Windsor
Jefferson Park
Whittier
Westwood
Sunnyside


In [10]:
print(denver_venues.shape)
denver_venues.head()

(1260, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Auraria,39.7506,-105.0075,Ball Arena,39.748700,-105.007600,Basketball Stadium
1,Auraria,39.7506,-105.0075,Orangetheory Fitness,39.750981,-105.003716,Gym
2,Auraria,39.7506,-105.0075,Coohills,39.750764,-105.003832,French Restaurant
3,Auraria,39.7506,-105.0075,Confluence Park,39.754487,-105.008569,Park
4,Auraria,39.7506,-105.0075,Kimpton Hotel Born,39.752554,-105.002370,Hotel


In [11]:
denver_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Auraria,100,100,100,100,100,100
Bear Valley,4,4,4,4,4,4
Berkeley,90,90,90,90,90,90
Chaffee Park,31,31,31,31,31,31
Cherry Creek,100,100,100,100,100,100
Congress Park,40,40,40,40,40,40
Five Points,100,100,100,100,100,100
Hampden,35,35,35,35,35,35
Harvey Park,6,6,6,6,6,6


In [12]:
print('There are {} uniques categories.'.format(len(denver_venues['Venue Category'].unique())))

There are 237 uniques categories.


In [13]:
denver_onehot = pd.get_dummies(denver_venues[['Venue Category']], prefix="", prefix_sep="")
denver_onehot['Neighborhood'] = denver_venues['Neighborhood'] 
fixed_columns = [denver_onehot.columns[-1]] + list(denver_onehot.columns[:-1])
denver_onehot = denver_onehot[fixed_columns]
denver_grouped = denver_onehot.groupby('Neighborhood').mean().reset_index()
denver_grouped.head()

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Car Wash,Cheese Shop,Chinese Restaurant,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Eye Doctor,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trade School,Trail,Train Station,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Auraria,0.0,0.0,0.0,0.01,0.050000,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.010000,0.0,0.000000,0.00,0.020000,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.000000,0.01,0.00,0.0,0.01,0.0,0.000000,0.000000,0.010000,0.030000,0.00,0.01,0.01,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.0,0.00,0.01,0.0,0.01,0.010000,0.020000,0.050000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.00,0.01,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.000000,0.0,0.00,0.0,0.01,0.0,0.0,0.01,0.000000,0.0,0.01,0.0,0.010000,0.02,0.000000,0.0,0.0,0.0,0.01,0.00,0.000000,0.0,0.000000,

In [14]:
num_top_venues = 10

for hood in denver_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = denver_grouped[denver_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Auraria----
                 venue  freq
0  American Restaurant  0.05
1                Hotel  0.05
2          Coffee Shop  0.05
3   Mexican Restaurant  0.03
4              Brewery  0.03


----Bear Valley----
         venue  freq
0         Park  0.50
1         Pool  0.25
2       Office  0.25
3  Zoo Exhibit  0.00
4   Nail Salon  0.00


----Berkeley----
            venue  freq
0         Brewery  0.07
1     Coffee Shop  0.07
2            Park  0.06
3  Breakfast Spot  0.04
4     Pizza Place  0.04


----Chaffee Park----
                  venue  freq
0  Fast Food Restaurant  0.16
1     Convenience Store  0.10
2                  Park  0.06
3          Intersection  0.06
4        Discount Store  0.06


----Cherry Creek----
                 venue  freq
0       Cosmetics Shop  0.06
1       Clothing Store  0.05
2          Coffee Shop  0.05
3  American Restaurant  0.05
4   Mexican Restaurant  0.04


----Congress Park----
                venue  freq
0                 Bar  0.08
1         Coffee Sh

In [25]:
#denver_grouped.iloc[0].plot.hist(legend=False)
denver_venues[['Neighborhood']['Auraria']]

<>:2: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:2: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<ipython-input-25-ffb0bc714fb8>:2: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  denver_venues[['Neighborhood']['Auraria']]


TypeError: list indices must be integers or slices, not str

In [22]:
denver_grouped.describe() 

,Zoo Exhibit,ATM,Accessories Store,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Car Wash,Cheese Shop,Chinese Restaurant,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Eye Doctor,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trade School,Trail,Train Station,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.00

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = denver_grouped['Neighborhood']

for ind in np.arange(denver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(denver_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Auraria,American Restaurant,Hotel,Coffee Shop,Mexican Restaurant,Brewery,Seafood Restaurant,Park,New American Restaurant,Restaurant,Sushi Restaurant
1,Bear Valley,Park,Pool,Office,Zoo Exhibit,Nail Salon,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant
2,Berkeley,Brewery,Coffee Shop,Park,Breakfast Spot,Pizza Place,Mexican Restaurant,Bar,Italian Restaurant,Gift Shop,Salon / Barbershop
3,Chaffee Park,Fast Food Restaurant,Convenience Store,Park,Intersection,Discount Store,Grocery Store,Food,Cosmetics Shop,Fried Chicken Joint,Pizza Place
4,Cherry Creek,Cosmetics Shop,Clothing Store,Coffee Shop,American Restaurant,Mexican Restaurant,Furniture / Home Store,Italian Restaurant,Department Store,Burger Joint,Restaurant


In [18]:
# set number of clusters
kclusters = 6

denver_grouped_clustering = denver_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(denver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 3, 0, 0, 0, 3, 2, 0])

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

denver_merged = df

denver_merged = denver_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

denver_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,River North,Auraria,39.7506,-105.0075,0,American Restaurant,Hotel,Coffee Shop,Mexican Restaurant,Brewery,Seafood Restaurant,Park,New American Restaurant,Restaurant,Sushi Restaurant
1,Capitol Hill,Cherry Creek,39.7195,-104.9481,0,Cosmetics Shop,Clothing Store,Coffee Shop,American Restaurant,Mexican Restaurant,Furniture / Home Store,Italian Restaurant,Department Store,Burger Joint,Restaurant
2,Capitol Hill,Congress Park,39.7329,-104.9494,0,Bar,Coffee Shop,Yoga Studio,Italian Restaurant,Grocery Store,Pizza Place,Breakfast Spot,Botanical Garden,Cosmetics Shop,Café
3,Lakeside,West Highland,39.7647,-105.0407,0,Coffee Shop,Mexican Restaurant,Pizza Place,Yoga Studio,Bar,Ice Cream Shop,Liquor Store,Sushi Restaurant,Brewery,Italian Restaurant
4,Lakeside,Sloan Lake,39.7486,-105.0473,0,Fast Food Restaurant,Coffee Shop,Ice Cream Shop,Bar,Park,Convenience Store,Burger Joint,Spa,Mobile Phone Shop,Sandwich Place


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denver_merged['Latitude'], denver_merged['Longitude'], denver_merged['Neighborhood'], denver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
clus1=denver_merged.loc[denver_merged['Cluster Labels'] == 0, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus1

In [ ]:
clus2=denver_merged.loc[denver_merged['Cluster Labels'] == 1, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus2

In [ ]:
clus3=denver_merged.loc[denver_merged['Cluster Labels'] == 2, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus3

In [ ]:
clus4=denver_merged.loc[denver_merged['Cluster Labels'] == 3, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus4

In [ ]:
clus5=denver_merged.loc[denver_merged['Cluster Labels'] == 4, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus5